In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pathlib import Path
from IPython.display import display, HTML
from Bio import SeqIO
import os
#from astropy import units as u
import sys
from readpaf import parse_paf
from collections import Counter
from tqdm import tqdm

import sv

#from itables import init_notebook_mode
#init_notebook_mode(all_interactive=True)

def to_latex(df, data, refname):
    latex = ""
    df.index = df.index.map(lambda x: f'\\{x}')
    df.columns = df.columns.str.replace(' ', '\\\\')
    df.columns = df.columns.str.replace('%', '\%')
    df.columns = df.columns.map(lambda x: '\makecell{' + x + '}')
    #df = df.astype(str).map(lambda x: x.rstrip('0').rstrip('.') if '.' in x else x)
    latex += df.to_latex(escape=False, label=f'tab:{refname}', caption=data, float_format = lambda x: '{:0.2f}'.format(x) if pd.notna(x) else '-')
    #latex += df.to_latex(float_format = lambda x: '{:0.2f}'.format(x) if pd.notna(x) else '-')
    latex += '\n'
    return latex

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def union_length(df):
    df_sorted = df.sort_values(by='target_start').reset_index(drop=True)
    merged_intervals = []
    current_start, current_end = df_sorted.iloc[0]['target_start'], df_sorted.iloc[0]['target_end']

    for index, row in df_sorted.iterrows():
        if index == 0:
            continue
        if row['target_start'] <= current_end:
            current_end = max(current_end, row['target_end'])
        else:
            merged_intervals.append((current_start, current_end))
            current_start, current_end = row['target_start'], row['target_end']

    merged_intervals.append((current_start, current_end))
    union_length = sum(end - start for start, end in merged_intervals)
    return union_length

In [11]:
def perc(a, b):
    if b == 0:
        return np.nan
    return 100.0 * a / b

def fasta2df(fn):
    seqs = SeqIO.parse(fn, "fasta")
    df = pd.DataFrame((str(s.id), str(s.seq)) for s in seqs)
    df.columns = ["ID", "Sequence"]
    return df

def is_overlapping(a, sv_row):
    return a.GT_from <= sv_row['END'] and sv_row['POS'] <= a.GT_to 
    
min_overlap = 0.9

# def is_correct(a):
#     if a.GT_ref != a.target_name:
#         return False
#     if a.GT_strand != a.strand:
#         return False
#     union_from = min(a.GT_from, a.target_start)
#     union_to = max(a.GT_to, a.target_end)

#     intersect_from = max(a.GT_from, a.target_start)
#     intersect_to = min(a.GT_to, a.target_end)
#     overlaps = intersect_to - intersect_from >= min_overlap * (union_to - union_from)
#     return overlaps

def is_correct_labels(a, GT_l, target_l, debug=False):
    if a.GT_ref != a.target_name:
        return False
    if a.GT_strand != a.strand:  # won't work for inversions
        return False
#    union_from = min(a.GT_from, a.target_start)
#    union_to = max(a.GT_to, a.target_end)

    GT_labels = Counter(target_l[a.GT_ref][a.GT_from:a.GT_to])
    target_labels = Counter(GT_l[a.target_name][a.target_start:a.target_end])
    #GT_labels = Counter(GT_l[a.GT_ref][a.GT_from:a.GT_to])
    #target_labels = Counter(target_l[a.target_name][a.target_start:a.target_end])
    intersection = GT_labels & target_labels
    union = GT_labels | target_labels
    overlaps = sum(intersection.values()) >= min_overlap * sum(union.values())

    if debug:
        display(a)
        display('           GT from {} to {}'.format(a.GT_from, a.GT_to))
        display('       target from {} to {}'.format(a.target_start, a.target_end))
        display('    GT_labels from {} to {}'.format(min(GT_labels), max(GT_labels)))
        display('target_labels from {} to {}'.format(min(target_labels), max(target_labels)))
        display('{} >?= {} = {} * union {} => {}'.format(sum(intersection.values()), min_overlap * sum(union.values()), min_overlap, sum(union.values()), overlaps))
    return overlaps

#def read_falls_on_what_sv_old(a, vcf_df):
#    #print('read_falls_on_what_sv', a)
#    sv_cnt = 0
#    sv_type = 'none'
#    for index, sv_row in vcf_df.iterrows():
#        if sv_row['CHROM'] != a.GT_ref:
#            continue
#        if is_overlapping(a, sv_row):
#            sv_cnt += 1
#            sv_type = sv_row['SVTYPE']
#    if sv_cnt > 1:
#        return 'multi'
#    return sv_type 

def read_falls_on_what_sv(a, vcf_df):
    query_start = a['GT_from']
    query_end = a['GT_to']
    start_idx = vcf_df['POS'].searchsorted(query_start, side='right')
    end_idx = vcf_df['END'].searchsorted(query_end, side='left')
    overlap_df = vcf_df.iloc[start_idx:end_idx]
    overlap_ch_df = overlap_df[overlap_df['CHROM'] == a['GT_ref']]
    overlap_ch_sv = overlap_ch_df[(overlap_ch_df['POS'] <= query_end) & (overlap_ch_df['END'] >= query_start)]
    #if len(overlap_ch_sv) > 0:
    #    display(a, overlap_ch_sv)
    if len(overlap_ch_sv) == 0:
        return 'none'
    elif len(overlap_ch_sv) == 1:
        return overlap_ch_sv.iloc[0]['SVTYPE']
    else:
        return 'multi'

def read_paf(pref, reads, experiment, tool, orig_l: dict, mutated_l: dict, vcf_df: pd.DataFrame):
    display(vcf_df)
    paf_file = pref.with_suffix('.paf')
    no_GT = False
    if not paf_file.exists():
        raise Exception(f"File does not exist or is empty: {paf_file}")
    with open(paf_file) as handle:
        df = parse_paf(handle, dataframe=True)
        df['experiment'] = experiment
        df['tool'] = tool
        try:
            df[ ['read_name', 'GT_ref', 'GT_from', 'GT_to', 'GT_strand'] ] = df['query_name'].str.split('!', expand=True)
            df['GT_from'] = df['GT_from'].astype(int)
            df['GT_to'] = df['GT_to'].astype(int)
            #df['is_correct_labels'] = df.apply(lambda x: is_correct_labels(x, orig_l, mutated_l), axis=1)
            #df['is_correct'] = df.apply(is_correct, axis=1)
            df['is_correct'] = df.apply(lambda x: is_correct_labels(x, orig_l, mutated_l), axis=1)
            df['start_diff'] = df.target_start - df.GT_from  # TODO: different coordinate systems!
            df['end_diff'] = df.target_end - df.GT_to  # TODO: different coordinate systems!
            df['read_sv'] = df.apply(lambda x: read_falls_on_what_sv(x, vcf_df), axis=1)
        except ValueError as e:
            display(e)
            df['read_name'] = df['query_name']
            #df['is_correct_labels'] = True
            df['is_correct'] = True
            df['start_diff'] = 0
            df['end_diff'] = 0
            df['read_sv'] = 'none'
            no_GT = True
        display(df)
    df = df.sort_values(['read_name', 'residue_matches'], ascending=[True, False], ignore_index=True)

    # initialize from left to right
    myNA = 'N/A'
    bad = 'bad'
    unique = 'uniq'
    multi = 'mult'
    bad_mappings_perc = bad+' mappings p.multiread'
    paf = {}
    paf[unique] = 0
    paf[multi] = 0
    paf['unmapped'] = 0
    for suffix in ['', ' Q60']:
        paf[bad + suffix] = 0
        if suffix == '':
            paf[multi+' '+bad+' group'] = 0
            paf[bad_mappings_perc] = 0
        paf[bad + ' ' + unique + suffix] = 0
        paf[bad + ' ' + multi + suffix] = 0
    prev_read_name = None

    at_least_one_correct = False
    for index, a in df.iterrows():
        assert(df.loc[index, 'read_name'] == a.read_name)
        next_read_name = df.loc[index + 1, 'read_name'] if index + 1 < len(df) else None
        is_unique = a.read_name != prev_read_name and a.read_name != next_read_name
        if is_unique:
            paf[unique] += 1
        if a.read_name != prev_read_name: 
            at_least_one_correct = False
        if a.is_correct:
            at_least_one_correct = True
        elif not is_unique:
            paf[bad_mappings_perc] += 1
        if a.read_name != next_read_name:
            if not at_least_one_correct and not is_unique:
                paf[multi+' '+bad+' group'] += 1
        if a.read_name == prev_read_name:
            continue
        if not is_unique:
            paf[multi] += 1
        if not a.is_correct:
            #print('wrong:', a)
            suffix = ' Q60' if a.mapping_quality == 60 else ''
            paf[bad + suffix] += 1
            if next_read_name != a.read_name:
                paf[bad + ' ' + unique + suffix] += 1
            else:
                paf[bad + ' ' + multi + suffix] += 1
        prev_read_name = a.read_name

    if paf[multi] > 0:
        paf[bad_mappings_perc] = '{:.2f}'.format(paf[bad_mappings_perc] / paf[multi])
    else:
        paf[bad_mappings_perc] = myNA
    paf['unmapped'] = reads - paf[unique] - paf[multi]

    paf['mean start diff'] = '{:.1f}'.format(df[df.is_correct].start_diff.mean())
    paf['mean end diff'] = '{:.1f}'.format(df[df.is_correct].end_diff.mean())

    for key in [unique, multi, 'unmapped', bad]:
        paf[key] = '{} ({:.2f}%)'.format(paf[key], perc(paf[key], reads))
    
    if no_GT:
        for suff in ['', ' '+unique, ' '+multi]:
            for suffsuff in ['', ' Q60']:
                paf[bad + suff + suffsuff] = myNA 
            paf[multi+' '+bad+' group'] = myNA
            paf['addit '+bad+' aligns'] = myNA
        paf['mean start diff'] = myNA
        paf['mean end diff'] = myNA

    paf['alignments depth'] = '{:.2f}'.format(sum(df['query_length']) / df.iloc[0]['target_length'])
    union_len = union_length(df)
    paf['covered reference'] = '{} ({:.2f}%)'.format(union_len, perc(union_len, df.iloc[0]['target_length']))

    # for each sv type in vcf_df or 'multi', count how many alignment are not correct and calculate their percentage of all alignments on this SV
    for sv_type in list(vcf_df['SVTYPE'].unique()) + ['multi']:
        if sv_type == 'none':
            continue
        sv_df = df[df['read_sv'] == sv_type]
        bad_sv = sv_df[~sv_df['is_correct']]
        paf[f'bad {sv_type}'] = '{} / {} ({:.2f}%)'.format(len(bad_sv), len(sv_df), perc(len(bad_sv), len(sv_df)))

    # debug
    #for row in df[df['read_sv'] != 'none'].itertuples():
    #    is_correct_labels(row, orig_l, mutated_l, debug=True)

    return pd.Series(paf, dtype='object')
    
#def read_eval(pref, reads):
#    eval_file = pref.with_suffix('.eval')
#    if not eval_file.exists() or os.stat(eval_file).st_size == 0:
#        raise Exception(f"File does not exist or is empty: {eval_file}")
#    evaldf = pd.read_csv(eval_file, sep='\t', names=['Q', 'quality', 'aligned', 'misaligned', 'misaligned/aligned', 'aligned cum'])
#    evals = {}
#    evals['eval aligned'] = evaldf.iloc[-1]['aligned cum']
#    evals['eval misaligned'] = evaldf.iloc[-1]['misaligned']
#    #display(f'aligned: {evals["eval aligned"]}, misaligned: {evals["eval misaligned"]}')
#    q60row = evaldf[evaldf['quality'] == 60]
#    assert(len(q60row) == 1)
#    #evals['eval unaligned'] = reads - evals['eval aligned']
#    evals['eval correct'] = evals['eval aligned'] - evals['eval misaligned']
#    evals['eval Q60 aligned'] = q60row['aligned'].values[0]
#    evals['eval Q60 misaligned'] = q60row['misaligned'].values[0]
#    #evals['eval noncorrect%'] = (100.0 * (evals['eval misaligned'] + evals['eval unaligned']) / evals['eval aligned']) #.round(2)
#    return pd.Series(evals, dtype='object')

def read_times(pref):
    times = {}
    with open(str(pref) + '.index.time') as f_index_time:
        index_time, index_mem = map(float, f_index_time.readline().split())
        times['index time'] = index_time #* u.second
        #times['index_mem'] = index_mem / 2**20
        with open(str(pref) + '.time') as f_time:
            total_time, total_mem = map(float, f_time.readline().split())
            #times['time total'] = total_time #* u.second
            times['map time'] = total_time - times['index time']
            times['memory'] = (total_mem / 2**20) #* u.GB
    return pd.Series(times, dtype='object').map('{:.1f}'.format)

def get_comparison_table(refname, readsim_refname, experiment, tools):
    empty_cell = -1
    alldf = pd.DataFrame()
    alldf.name = experiment
    ref = fasta2df(Path('refs') / (refname+'.fa'))
    reads = fasta2df(Path('reads') / Path(experiment+'.fa'))

    # SVs
    orig_fa_dict = sv.read_fasta_file(Path('refs') / (refname+'.fa'))
    orig_l = sv.gen_unique_labels(orig_fa_dict)
    try:
        vcf_df = sv.read_vcf(Path('refs') / (readsim_refname+'.vcf'))
        assert((vcf_df.POS <= vcf_df.END).all())
        vcf_df = vcf_df.sort_values(by='POS').reset_index(drop=True)
        mutated_fa, mutated_l = sv.mutate(orig_fa_dict, orig_l, vcf_df)
    except FileNotFoundError:
        vcf_df = None
        mutated_fa, mutated_l = orig_fa_dict, orig_l

    rows = []

    for tool in tqdm(tools, desc=f'Tools for {experiment}'):
        d = Path("out") / experiment / tool / tool
        row = pd.Series({
            'tool': tool,
            'reads': len(reads),
        })
        row = pd.concat([row, read_paf(d, len(reads), experiment, tool, orig_l, mutated_l, vcf_df)])     # .paf
        try:
            row = pd.concat([row, read_times(d)])   # .time, .index.time
        except Exception as e:
            print(f"An error occurred while reading times {d}: {e}")
            row['index time'] = empty_cell
            row['map time'] = empty_cell
            row['memory'] = empty_cell
        rows.append(row)
    alldf = pd.DataFrame(rows)
    alldf = alldf.set_index('tool')
    alldf.index.name = None
    return alldf

Build a table to compare the mappers by accuracy, runtime (indexing and mapping) and memory.

In [10]:
tools = ['sweepmap', 'sweepmap-slow', 'mapquik', 'blend', 'minimap', 'winnowmap'] 
#tools = ['winnowmap', 'minimap'] 
experiments = [
#    ('t2tChrY', 't2tChrY', 't2tChrY-readst2tChrY-a0.99-d10-l10000'),
#    ('chm13',   'chm13',   'chm13-readschm13-a0.99-d0.1-l10000'),
#    ('t2tChrY', 't2tChrY', 't2tChrY-readst2tChrY-a0.99-d1-l24000'),
#    ('chm13',   'chm13',   'HG002_24kb'),
	('t2tChrY', 't2tChrY-SVs', 't2tChrY-readst2tChrY-SVs-a0.99-d0.1-l10000'),
]

pd.set_option('display.width', 100)
css = """ <style> table { font-family: "Courier New", Courier, monospace; } </style> """
display(HTML(css))
dfs = []
keys = []
for refname, readsim_refname, data in experiments:
    vcf_df = get_comparison_table(refname=refname, readsim_refname=readsim_refname, experiment=data, tools=tools).round(2)
    dfs.append(vcf_df)
    keys.append(data)
    df_styled = vcf_df.style.set_caption(data)
    display(df_styled)
    #print(to_latex(df))
#DF = pd.concat(dfs, keys=keys)
#display(DF)

Tools for t2tChrY-readst2tChrY-SVs-a0.99-d0.1-l10000:   0%|          | 0/2 [00:00<?, ?it/s]

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,PRECISE,SVTYPE,SVMETHOD,CHR2,END,SVLEN,dup_num
0,NC_060948.1,5299,[INV2156SURVIVOR],N,INV,None,PASS,True,INV,SURVIVOR_sim,NC_060948.1,6011,712,NaN
1,NC_060948.1,46009,[INV2230SURVIVOR],N,INV,None,PASS,True,INV,SURVIVOR_sim,NC_060948.1,46722,713,NaN
2,NC_060948.1,73059,[DEL1089SURVIVOR],TGACAGGAAGAGGGGGTCTCCGGCTGTGACAGGAAGAGGGAGTCTC...,T,None,PASS,True,DEL,SURVIVOR_sim,NC_060948.1,73242,183,NaN
3,NC_060948.1,91267,[DEL1921SURVIVOR],TCAGCTTCGGAGACACAATTCTCTATCCTTTTCATGTCATTTTGCT...,T,None,PASS,True,DEL,SURVIVOR_sim,NC_060948.1,91319,52,NaN
4,NC_060948.1,94881,[DUP206SURVIVOR],N,DUP,None,PASS,True,DUP,SURVIVOR_sim,NC_060948.1,95279,398,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,NC_060948.1,62340189,[DUP973SURVIVOR],N,DUP,None,PASS,True,DUP,SURVIVOR_sim,NC_060948.1,62340741,552,3
2996,NC_060948.1,62410894,[DUP791SURVIVOR],N,DUP,None,PASS,True,DUP,SURVIVOR_sim,NC_060948.1,62411154,260,5
2997,NC_060948.1,62426699,[DUP225SURVIVOR],N,DUP,None,PASS,True,DUP,SURVIVOR_sim,NC_060948.1,62427269,570,4
2998,NC_060948.1,62449565,[DUP546SURVIVOR],N,DUP,None,PASS,True,DUP,SURVIVOR_sim,NC_060948.1,62449846,281,3


,query_name,query_length,query_start,query_end,strand,target_name,target_length,target_start,target_end,residue_matches,...,tool,read_name,GT_ref,GT_from,GT_to,GT_strand,is_correct,start_diff,end_diff,read_sv
0,S1_264!NC_060948.1!863842!885058!-,21263,27,21246,-,NC_060948.1,62460029,837522,858694,8470,...,winnowmap,S1_264,NC_060948.1,863842,885058,-,True,-26320,-26364,multi
1,S1_212!NC_060948.1!59000472!59020548!-,20138,51,19968,-,NC_060948.1,62460029,57054525,57074038,5757,...,winnowmap,S1_212,NC_060948.1,59000472,59020548,-,True,-1945947,-1946510,DEL
2,S1_388!NC_060948.1!54641776!54659240!-,17502,2,17464,-,NC_060948.1,62460029,52822468,52839892,4933,...,winnowmap,S1_388,NC_060948.1,54641776,54659240,-,True,-1819308,-1819348,multi
3,S1_578!NC_060948.1!1612770!1630118!-,17383,10,17366,-,NC_060948.1,62460029,1567416,1582682,6495,...,winnowmap,S1_578,NC_060948.1,1612770,1630118,-,False,-45354,-47436,none
4,S1_49!NC_060948.1!22850635!22867945!+,17373,12,17311,+,NC_060948.1,62460029,22055903,22073141,6997,...,winnowmap,S1_49,NC_060948.1,22850635,22867945,+,True,-794732,-794804,none
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
767,S1_393!NC_060948.1!58608478!58614092!+,5632,37,5631,+,NC_060948.1,62460029,56670599,56676175,1673,...,winnowmap,S1_393,NC_060948.1,58608478,58614092,+,True,-1937879,-1937917,none
768,S1_395!NC_060948.1!31001437!31007041!+,5610,124,5579,+,NC_060948.1,62460029,29942493,29947942,1586,...,winnowmap,S1_395,NC_060948.1,31001437,31007041,+,True,-1058944,-1059099,none
769,S1_465!NC_060948.1!53771016!53776522!+,5526,36,5526,+,NC_060948.1,62460029,51986505,51991975,1559,...,winnowmap,S1_465,NC_060948.1,53771016,53776522,+,True,-1784511,-1784547,none
770,S1_148!NC_060948.1!17603290!17608747!-,5470,13,5403,-,NC_060948.1,62460029,16970430,16975806,2295,...,winnowmap,S1_148,NC_060948.1,17603290,17608747,-,True,-632860,-632941,none


Pandas(Index=3, query_name='S1_101!NC_060948.1!59546000!59555296!+', query_length=9302, query_start=5, query_end=9280, strand='+', target_name='NC_060948.1', target_length=62460029, target_start=57575157, target_end=57584426, residue_matches=2906, alignment_block_length=9296, mapping_quality=3, tp='P', cm=220, s1=2905, s2=2863, rl=1118, experiment='t2tChrY-readst2tChrY-SVs-a0.99-d0.1-l10000', tool='winnowmap', read_name='S1_101', GT_ref='NC_060948.1', GT_from=59546000, GT_to=59555296, GT_strand='+', is_correct=True, start_diff=-1970843, end_diff=-1970870, read_sv='INS')

'           GT from 59546000 to 59555296'

...

'    GT_labels from 41752032 to 41763706'

'target_labels from 41752272 to 41763548'

'11277 >?= 10507.5 = 0.9 * union 11675 => True'

Tools for t2tChrY-readst2tChrY-SVs-a0.99-d0.1-l10000: 100%|██████████| 2/2 [00:14<00:00,  7.15s/it]


,reads,uniq,mult,unmapped,bad,mult bad group,bad mappings p.multiread,bad uniq,bad mult,bad Q60,bad uniq Q60,bad mult Q60,mean start diff,mean end diff,alignments depth,covered reference,bad INV,bad DEL,bad DUP,bad INS,bad multi,index time,map time,memory
winnowmap,638,572 (89.66%),66 (10.34%),0 (0.00%),78 (12.23%),54,2.85,52,26,53,25,28,-1064044.2,-1064158.5,0.12,5875957 (9.41%),16 / 64 (25.00%),16 / 33 (48.48%),15 / 64 (23.44%),15 / 43 (34.88%),15 / 49 (30.61%),2.7,689.2,0.5
minimap,638,557 (87.30%),81 (12.70%),0 (0.00%),105 (16.46%),59,3.27,53,52,23,16,7,-1044700.3,-1044910.7,0.14,5763502 (9.23%),27 / 75 (36.00%),29 / 48 (60.42%),18 / 69 (26.09%),11 / 40 (27.50%),29 / 64 (45.31%),1.6,12.1,0.3


Visualize how SweepMap parameters influence its accuracy and mapping runtime.

In [67]:
Ks = [14, 16, 18, 20, 22, 24, 26]
Rs = ['0.01', '0.05', '0.1', '0.15', '0.2']

def read_matrix(Ys, Xs, ylabel, xlabel, experiment):
    TOOL = 'sweepmap'
    DATA = ('t2tChrY', 't2tChrY-a0.99-d1-l10000') # ('chm13', 'chm13-a0.99-d1-l10000')

    VARS = {'thinning': ('S', 'M'), 'sketching': ('K', 'R')}
    vary, varx = VARS[experiment]

    map_time_df = pd.DataFrame(index=Ys, columns=Xs)
    map_time_df.index.name = ylabel
    map_time_df.columns.name = xlabel
    issue_perc_df = map_time_df.copy() # pd.DataFrame(index=MAX_SEEDS, columns=MAX_MATCHES)

    for y in Ys:
        for x in Xs:
            refname, data = DATA
            reads = fasta2df(Path('reads') / Path('reads-'+data+'.fa'))
            pref = Path("out") / data / experiment / f'{TOOL}-{vary}{y}-{varx}{x}'
            eval_file = str(pref) + '.eval'
            unaligned,  aligned,  wrong,     issue_perc = get_eval(eval_file, len(reads))
            index_time, map_time, index_mem, total_mem  = get_times(pref)
            map_time_df.loc[y, x] = map_time
            issue_perc_df.loc[y, x] = issue_perc / 100.0

    return map_time_df, issue_perc_df

def plot_SM_heatmap(df, name, title, cm):
    fig, ax = plt.subplots(figsize=(6, 4))
    sns.heatmap(df.astype(float), ax=ax, annot=True, fmt=".2f", cmap=cm) #, cbar=False)
    ax.set_title(title)
    ax.set_ylabel(df.index.name)
    ax.set_xlabel(df.columns.name)
    ax.set_yticklabels(ax.get_yticklabels(), rotation=0)
    ax.invert_yaxis()
    plt.tight_layout()
    plt.show()
    fig.savefig(f'imgs/{name}.png', dpi=300)
    plt.close(fig)

KR_map_time_df, KR_issue_perc_df = read_matrix(Ks, Rs, 'kmer size (K)', 'FracMinHash ratio (R)', 'sketching')
plot_SM_heatmap(KR_map_time_df, name='heatmapKR_maptime', title='Map time', cm='mako_r')
plot_SM_heatmap(KR_issue_perc_df, name='heatmapKR_percissues', title='Percent of issues (wrong + unaligned)', cm='rocket_r')

MAX_SEEDS = [10, 30, 100, 300, 1000, 3000, 10000]
MAX_MATCHES = [100, 300, 1000, 3000, 10000, 30000, 100000, 300000]
SM_map_time_df, SM_issue_perc_df = read_matrix(MAX_SEEDS, MAX_MATCHES, 'max seeds (S)', 'max matches (M)', 'thinning')
plot_SM_heatmap(SM_map_time_df, name='heatmapSM_maptime', title='Map time', cm='mako_r')
plot_SM_heatmap(SM_issue_perc_df, name='heatmapSM_percissues', title='Percent of issues (wrong + unaligned)', cm='rocket_r')

FileNotFoundError: [Errno 2] No such file or directory: 'reads/reads-t2tChrY-a0.99-d1-l10000.fa'

In [ ]:
vcf_df = read_vcf('refs/t2tChrY-SVs.vcf')
fa_orig = read_fasta_file('refs/t2tChrY.fa')
fa_SURVIVOR = read_fasta_file('refs/t2tChrY-SVs.fasta')
l_orig = gen_unique_labels(fa_orig)
fa_SV, labels_SV = mutate(fa_orig, l_orig, vcf_df)
#are_equal(fa_orig, fa_SURVIVOR)
are_equal(fa_SV, fa_SURVIVOR)

/home/pesho/.local/lib/python3.10/site-packages/vcfpy/header.py:413: FieldInfoNotFound: INFO dup_num not found using String/'.' instead
  warnings.warn(


,CHROM,POS,ID,REF,ALT,QUAL,FILTER,PRECISE,SVTYPE,SVMETHOD,CHR2,END,SVLEN,dup_num
4,NC_060948.1,3430510,[INS4SURVIVOR],A,GGAGCTTAGGAGGTAAAAGGAACTTGTGAAGGTGAACTATCCCTAC...,None,PASS,True,INS,SURVIVOR_sim,NC_060948.1,3430745,235,NaN
2,NC_060948.1,14398344,[DUP2SURVIVOR],N,DUP,None,PASS,True,DUP,SURVIVOR_sim,NC_060948.1,14398656,312,5
5,NC_060948.1,15092668,[DEL5SURVIVOR],TTAGACTTCCTAAATATATAAAGCAAATATTAATGGACATAAAGGG...,T,None,PASS,True,DEL,SURVIVOR_sim,NC_060948.1,15092864,196,NaN
8,NC_060948.1,16086589,[INV8SURVIVOR],N,INV,None,PASS,True,INV,SURVIVOR_sim,NC_060948.1,16087210,621,NaN
1,NC_060948.1,27847166,[DUP1SURVIVOR],N,DUP,None,PASS,True,DUP,SURVIVOR_sim,NC_060948.1,27847906,740,6
0,NC_060948.1,35790329,[DUP0SURVIVOR],N,DUP,None,PASS,True,DUP,SURVIVOR_sim,NC_060948.1,35790540,211,2
7,NC_060948.1,42134928,[INV7SURVIVOR],N,INV,None,PASS,True,INV,SURVIVOR_sim,NC_060948.1,42135682,754,NaN
3,NC_060948.1,47095939,[INS3SURVIVOR],G,AGAACCCCCCTTTAATATGAGCGAAATGCCTCTACCCTGGACCACG...,None,PASS,True,INS,SURVIVOR_sim,NC_060948.1,47096432,493,NaN
6,NC_060948.1,56548921,[INV6SURVIVOR],N,INV,None,PASS,True,INV,SURVIVOR_sim,NC_060948.1,56549555,634,NaN


CHROM                                             NC_060948.1
POS                                                   3430510
ID                                             [INS4SURVIVOR]
REF                                                         A
ALT         GGAGCTTAGGAGGTAAAAGGAACTTGTGAAGGTGAACTATCCCTAC...
QUAL                                                     None
FILTER                                                   PASS
PRECISE                                                  True
SVTYPE                                                    INS
SVMETHOD                                         SURVIVOR_sim
CHR2                                              NC_060948.1
END                                                   3430745
SVLEN                                                     235
dup_num                                                   NaN
Name: 4, dtype: object

2. add 0:3430510 of len 3430510
4. ins of len 235


CHROM          NC_060948.1
POS               14398344
ID          [DUP2SURVIVOR]
REF                      N
ALT                    DUP
QUAL                  None
FILTER                PASS
PRECISE               True
SVTYPE                 DUP
SVMETHOD      SURVIVOR_sim
CHR2           NC_060948.1
END               14398656
SVLEN                  312
dup_num                  5
Name: 2, dtype: object

...

CHROM          NC_060948.1
POS               56548921
ID          [INV6SURVIVOR]
REF                      N
ALT                    INV
QUAL                  None
FILTER                PASS
PRECISE               True
SVTYPE                 INV
SVMETHOD      SURVIVOR_sim
CHR2           NC_060948.1
END               56549555
SVLEN                  634
dup_num                NaN
Name: 6, dtype: object

2. add 47095939:56548921 of len 9452982
6. inv 56548921:56549555 of len 634


CHROM    !!fake_chrom!!
dtype: object

1. add 56549555: of len 5910474
62466983 62466983


True